<a href="https://colab.research.google.com/github/IKeeso/Project9_data/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stanza
!pip install conllu
!git clone https://github.com/IKeeso/Project9_data.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 14.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [2]:
import stanza
from stanza.utils.conll import CoNLL
from conllu import parse
from conllu import TokenList
import re
import os

!ls Project9_data


'Allmän screening för autism kan bli realitet vid 2,5-årsbesöket BVC i nyckelposition för tidig diagnos – om studierna håller, Staffan Janson, Margareta Blennow.txt'
'Autismens historia – en internationell  forskningsöversiktSofia Littmarck, Peter Skagius & Karin Zetterqvist Nelson.txt'
'Autism och oxytocin – aktuellt forskningsfält, Anders Hansen.txt'
'Autismspektrumtillstånd vid svår CP upptäcks ofta sent Retrospektiv genomgång av 10 års journaler, Margareta Dahl, Christina Bergsten, Anita Hammarberg, Marie Sandström, Margareta Strinnholm.txt'
'autisterna om kvinnor på spektrat.conllu'
'Autisterna Om kvinnor på spektrat.txt'
 avhandlingar
 Blogs
 blogstestfiles
'BVC_s unika roll – att fånga upp autism tidigt Erfarenheter av nya rutiner i barn hälsovården i Göteborg , GUDRUN NYGREN, EVA SANDBERG, THOMAS ARVIDSSON, CHRISTOPHER GILLBERG,.txt'
'CONLLU TEXTS'
'en annorlunda barndom.conllu'
'En annorlunda barndom.txt'
'ett annorlunda liv.conllu'
'Ett annorlunda liv.txt'
'Forskning med juri

In [3]:
stanza.download('sv')
nlp = stanza.Pipeline(lang='sv')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: sv (Swedish) ...


INFO:stanza:Downloaded file to /root/stanza_resources/sv/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: sv (Swedish):
| Processor | Package            |
----------------------------------
| tokenize  | talbanken          |
| pos       | talbanken_charlm   |
| lemma     | talbanken_nocharlm |
| depparse  | talbanken_charlm   |
| ner       | suc3shuffle        |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [4]:
def tag_and_convert(filename):
  %cd /content/Project9_data
  try:
    with open(filename, 'r', encoding='utf-8') as file:
      text = file.read()
      file.close()
  except:
    text = filename
  doc = nlp(text)
  pathname = filename.split(".")[0]
  conllufilename = f"{pathname}.conllu".lower()
  CoNLL.write_doc2conll(doc, conllufilename)

  return conllufilename

In [5]:
class SemanticAnalyzer:
  def __init__(self, file_path, lang):
    self.lang = lang
    self.file_path = file_path
    self.data = self.parse_conll_u(self.file_path)
    if lang == "pfl":
      self.PFL_sentence_matches = self.person_first_lang(self.data)
    elif lang == "ifl":
      self.IFL_sentence_matches = self.identity_first_lang(self.data)

  def parse_conll_u(self, file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
      data = {}
      sentence = []
      for line in file:
        line = line.strip()
        if line.startswith('# text'):
          sentence = []
        if line.startswith("# sent_id"):
          line = line.split("=")
          key = line[1].strip()
          sentence = []
        elif line == '':
          if sentence:
            data[key] = sentence
        else:
            sentence.append(line.split('\t'))
    return data

  def person_first_lang(self, data):
    PFL_sents = {}
    PFL_regx = re.compile(r"ha|med", re.IGNORECASE)
    Human_regx = re.compile(r"^jag$|^mig$|^du$|^dig$|^din$|^ditt$|^dina$|^han$|^honom$|^hans$|^hon$|^henne$|^hennes$|^den$|^det$|^dess$|^vi$|^oss$|^våra$|^ni$|^er$|^ert$|^era$|^de$|^dem$|^deras$|^man$|^ens$|^hen$|^hens$|^sig$|^sin$|^sitt$|^sina$|^person$|^människa$|^kvinna$|^man$|^barn$|^pojke$|^flicka$|^bebis$|^ungdom$|^vuxen$|^åldring$|^student$|^elev$|^lärare$|^doktor$|^sjuksköterska$|^ingenjör$|^advokat$|^forskare$|^författare$|^musiker$|^artist$|^skådespelare$|^politiker$|^ledare$|^president$|^kung$|^drottning$|^prins$|^prinsessa$|^entreprenör$|^chef$|^arbetare$|^anställd$|^volontär$|^mor$|^far$|^förälder$|^son$|^dotter$|^bror$|^syster$|^farfar$|^farmor$|^morfar$|^mormor$|^barnbarn$|^kusin$|^farbror$|^faster$|^morbror$|^moster$|^läkare$|^jurist$|^polis$|^brandman$|^soldat$|^professor$|^arkitekt$|^konstnär$|^journalist$|^programmerare$|^ekonom$|^kock$|^servitör$|^vän$|^granne$|^kollega$|^partner$|^make$|^maka$|^pojkvän$|^flickvän$|^fiende$|^herr$|^fru$|^fröken$|^direktör$|^minister$|^talman$|^domare$|^vittne$|^misstänkt$|^fånge$|^klient$|^patient$|^kund$|^åhörare$|^medlem$|^ledamot$|^guide$|^tränare$|^idrottare$|^mäklare$|^chaufför$|^pilot$|^sjöman$|^kapten$|^reseledare$|^affärsman$|^modell$|^fotograf$|^bloggare$|^influencer$|^tekniker$|^mekaniker$|^snickare$|^elektriker$|^rörmokare$|^målare$|^bonde$|^fiskare$|^jägare$|^trädgårdsmästare$|^designer$|^rektor$|^dekan$|^prefekt$|^sekreterare$|^assistent$|^konsult$|^rådgivare$|^terapeut$|^psykolog$|^psykiater$|^sjukgymnast$|^dietist$|^logoped$|^optiker$|^tandläkare$|^barnmorska$|^undersköterska$|^bibliotikarie$|^administratör$|^kontorist$|^kassör$|^handläggare$|^arbetsledare$|^avdelningschef$|^vd$|^företagare$|^entreprenör$|^investerare$|^aktieägare$|^ägare$|^delägare$|^sambo$|^särbo$|^styvfar$|^styvmor$|^styvson$|^styvdotter$|^halvbror$|^halvsyster$|^änka$|^änkling$|^singel$|^älskare$|^älskarinna$|^adoptivfar$|^adoptivmor$|^adopterad$|^mentor$|^lärling$|^präst$|^pastor$|^biskop$|^munk$|^nunna$|^profet$|^imam$|^rabbin$|^sektledare$|^missionär$|^diakon$|^samarit$|^helgon$|^martyr$|^pilgrim$|^asket$|^eremit$|^vagabond$|^nomad$|^hemlös$|^flykting$|^invandrare$|^migrant$|^expat$|^turist$|^resenär$|^besökare$|^gäst$|^hyresgäst$|^värd$|^värdinna$|^inneboende$|^volontärarbetare$|^träningspartner$|^lagkamrat$|^motståndare$|^rival$|^allierad$|^medarbetare$|^teamledare$|^teammedlem$|^supervisor$|^handledare$|^praktikant$|^medlare$|^förmyndare$|^godman$|^beskyddare$|^sponsor$|^donator$|^filantrop$|^biståndsarbetare$|^aktivist$|^demonstrant$|^strejkande$|^åskådare$|^fan$|^följare$|^supporter$|^beundrare$|^idol$|^förebild$|^talesperson$|^representant$|^delegat$|^ombud$|^agent$|^kanslist$|^koordinator$|^projektledare$|^regionchef$|^platschef$|^butikschef$|^fastighetsägare$|^hyresvärd$|^revisor$|^bokhållare$|^redovisningsekonom$|^åklagare$|^försvarsadvokat$|^kriminalare$|^detektiv$|^spanare$|^inspektör$|^kommissarie$|^vakt$|^säkerhetsvakt$|^väktare$|^livvakt$|^dörrvakt$|^ordningsvakt$|^badvakt$|^ronderare$|^skyddsvakt$|^kontrollant$|^övervakare$|^kurator$|^coach$|^instruktör$|^läxhjälpare$|^karriärcoach$|^livscoach$|^livskamrat$|^brorson$|^systerson$|^systerdotter$|^brorsdotter$|^svärfar$|^svärmor$|^svärson$|^svärdotter$|^svåger$|^svägerska$|^fosterbarn$|^fosterfar$|^fostermor$|^fosterbror$|^fostersyster$|^närstående$|^bekant$|^medborgare$|^civilsamhällesaktör$|^företagsledare$|^folkvald$|^tjänsteman$|^diplomathustru$|^presidentkandidat$|^parlamentariker$|^hövding$|^stamledare$|^krigare$|^försvarare$|^fredsmäklare$|^förhandlare$|^lobbyist$|^talesman$|^presstalesman$|^PR-expert$|^kommunikatör$|^marknadsförare$|^konsultchef$|^säkerhetsrådgivare$|^brandingenjör$|^riskanalytiker$|^forskningsassistent$|^doktorand$|^magisterstudent$|^stipendiat$|^trainee$|^praktikhandledare$|^karriärvägledare$|^psykologiprofessor$|^sociolog$|^antropolog$|^arkivarie$|^kulturhistoriker$|^museichef$|^konservator$|^teolog$|^religionshistoriker$|^lingvist$|^språkvetare$|^översättare$|^tolk$|^fonetiker$|^logoped$|^retoriker$|^dramaturg$|^manusförfattare$|^regissör$|^filmproducent$|^biografägare$|^ljusdesigner$|^kostymör$|^scenograf$|^teaterkritiker$|^kulturjournalist$|^litteraturkritiker$|^recensent$|^bildredaktör$|^nyhetsankare$|^krönikör$|^spökskrivare$|^ledarskribent$|^debattör$|^talarcoach$|^pr-expert$|^eventkoordinator$|^värdepappersmäklare$|^börsanalytiker$|^försäkringsrådgivare$|^pensionsrådgivare$|^finansiellrådgivare$|^bankdirektör$|^kreditgivare$|^lånehandläggare$|^borgensman$|^panträttsanalytiker$|^fastighetsförvaltare$|^fastighetsmäklare$|^byggprojektledare$|^stadsplanerare$|^inredningsarkitekt$|^landskapsarkitekt$|^trädgårdsarkitekt$|^byggnadsingenjör$|^byggnadskonstruktör$|^fastighetsskötare$|^underhållstekniker$|^driftchef$|^energiexpert$|^hållbarhetsstrateg$|^klimatforskare$|^miljökonsult$|^ekosystemforskare$|^agronom$|^skogsforskare$|^växtbiolog$|^djurvårdare$|^veterinär$|^djursjukvårdare$|^djurskötare$|^djuruppfödare$|^zoopedagog$|^djurtränare$|^jockey$|^ryttare$|^ridlärare$|^hovslagare$|^hästskötare$|^dressyrryttare$|^galopptränare$|^agilitytränare$|^hundtränare$|^kattuppfödare$|^akvarist$|^reptiluppfödare$|^fågeluppfödare$|^fiskodlare$|^biodlare$|^honungsproducent$|^fruktodlare$|^grönsaksodlare$|^spannmålsbonde$|^spannmålsproducent$|^lantbrukare$|^jordbrukare$|^trädgårdsmästare$|^blomsterodlare$|^floristikexpert$|^blomsterhandlare$|^landskapsingenjör$|^parkförvaltare$|^parkvakt$|^trädgårdsarbetare$|^landskapsarkitekt$|^kyrkvaktmästare$|^kyrkogårdsvakt$|^krematoriearbetare$|^gravgrävare$|^ceremonimästare$|^begravningspräst$|^sorgekoordinator$|^bröllopskoordinator$|^vigselförrättare$|^festfixare$|^eventplanerare$|^konferensvärd$|^kongressorganisatör$|^mässarrangör$|^produktionsledare$|^produktchef$|^produktutvecklare$|^kvalitetsansvarig$|^innovationsledare$|^forskningschef$|^forskningsingenjör$|^laboratoriechef$|^laboratorieassistent$|^kemist$|^biokemist$|^molekylärbiolog$|^genetikforskare$|^bioinformatiker$|^biostatistiker$|^kliniskforskare$|^läkemedelsforskare$|^toxikolog$|^farmaceut$|^apotekare$|^receptarie$|^läkemedelskonsulent$|^läkemedelsinformatör$|^klinikchef$|^vårdavdelningschef$|^vårdadministratör$|^sjukhusdirektör$|^sjukvårdsplanerare$|^vårdkoordinator$|^operationssjuksköterska$|^operationsläkare$|^anestesiläkare$|^narkosläkare$|^narkossköterska$|^operationsassistent$|^akutsjuksköterska$|^akutläkare$|^intensivvårdssjuksköterska$|^intensivvårdsläkare$|^vårdpersonal$|^sjukhemschef$|^äldreomsorgschef$|^hemtjänstpersonal$|^personligassistent$|^funktionsstödsassistent$|^stödperson$|^anhörigvårdare$|^omsorgspersonal$|^dagispersonal$|^barnskötare$|^fritidspedagog$|^förskollärare$|^montessoripedagog$|^speciallärare$|^specialpedagog$", re.IGNORECASE)

    for id, sentence in data.items():
      for token in sentence:

        if sentence[int(token[6])-1][1] == "autism" or sentence[int(token[6])-1][1] == "autismspektrumtillstånd" or sentence[int(token[6])-1][1] == "ast" or sentence[int(token[6])-1][1] == "autismdiagnos":
          if PFL_regx.match(token[2]) and token[6] != "root" or token[9].split("|")[-1] != "ner=O":
            print(sentence)
            PFL_sents[id] = sentence

    return PFL_sents

  def identity_first_lang(self, data):
    IFL_sents = {}
    manual_sents = {}
    IFL_regx = re.compile(r"^.*(autistisk|autistiska)", re.IGNORECASE)
    Human_regx = re.compile(r"^person$|^människa$|^kvinna$|^man$|^barn$|^pojke$|^flicka$|^bebis$|^ungdom$|^vuxen$|^åldring$|^student$|^elev$|^lärare$|^doktor$|^sjuksköterska$|^ingenjör$|^advokat$|^forskare$|^författare$|^musiker$|^artist$|^skådespelare$|^politiker$|^ledare$|^president$|^kung$|^drottning$|^prins$|^prinsessa$|^entreprenör$|^chef$|^arbetare$|^anställd$|^volontär$|^mor$|^far$|^förälder$|^son$|^dotter$|^bror$|^syster$|^farfar$|^farmor$|^morfar$|^mormor$|^barnbarn$|^kusin$|^farbror$|^faster$|^morbror$|^moster$|^läkare$|^jurist$|^polis$|^brandman$|^soldat$|^professor$|^arkitekt$|^konstnär$|^journalist$|^programmerare$|^ekonom$|^kock$|^servitör$|^vän$|^granne$|^kollega$|^partner$|^make$|^maka$|^pojkvän$|^flickvän$|^fiende$|^herr$|^fru$|^fröken$|^direktör$|^minister$|^talman$|^domare$|^vittne$|^misstänkt$|^fånge$|^klient$|^patient$|^kund$|^åhörare$|^medlem$|^ledamot$|^guide$|^tränare$|^idrottare$|^mäklare$|^chaufför$|^pilot$|^sjöman$|^kapten$|^reseledare$|^affärsman$|^modell$|^fotograf$|^bloggare$|^influencer$|^tekniker$|^mekaniker$|^snickare$|^elektriker$|^rörmokare$|^målare$|^bonde$|^fiskare$|^jägare$|^trädgårdsmästare$|^designer$|^rektor$|^dekan$|^prefekt$|^sekreterare$|^assistent$|^konsult$|^rådgivare$|^terapeut$|^psykolog$|^psykiater$|^sjukgymnast$|^dietist$|^logoped$|^optiker$|^tandläkare$|^barnmorska$|^undersköterska$|^bibliotikarie$|^administratör$|^kontorist$|^kassör$|^handläggare$|^arbetsledare$|^avdelningschef$|^vd$|^företagare$|^entreprenör$|^investerare$|^aktieägare$|^ägare$|^delägare$|^sambo$|^särbo$|^styvfar$|^styvmor$|^styvson$|^styvdotter$|^halvbror$|^halvsyster$|^änka$|^änkling$|^singel$|^älskare$|^älskarinna$|^adoptivfar$|^adoptivmor$|^adopterad$|^mentor$|^lärling$|^präst$|^pastor$|^biskop$|^munk$|^nunna$|^profet$|^imam$|^rabbin$|^sektledare$|^missionär$|^diakon$|^samarit$|^helgon$|^martyr$|^pilgrim$|^asket$|^eremit$|^vagabond$|^nomad$|^hemlös$|^flykting$|^invandrare$|^migrant$|^expat$|^turist$|^resenär$|^besökare$|^gäst$|^hyresgäst$|^värd$|^värdinna$|^inneboende$|^volontärarbetare$|^träningspartner$|^lagkamrat$|^motståndare$|^rival$|^allierad$|^medarbetare$|^teamledare$|^teammedlem$|^supervisor$|^handledare$|^praktikant$|^medlare$|^förmyndare$|^godman$|^beskyddare$|^sponsor$|^donator$|^filantrop$|^biståndsarbetare$|^aktivist$|^demonstrant$|^strejkande$|^åskådare$|^fan$|^följare$|^supporter$|^beundrare$|^idol$|^förebild$|^talesperson$|^representant$|^delegat$|^ombud$|^agent$|^kanslist$|^koordinator$|^projektledare$|^regionchef$|^platschef$|^butikschef$|^fastighetsägare$|^hyresvärd$|^revisor$|^bokhållare$|^redovisningsekonom$|^åklagare$|^försvarsadvokat$|^kriminalare$|^detektiv$|^spanare$|^inspektör$|^kommissarie$|^vakt$|^säkerhetsvakt$|^väktare$|^livvakt$|^dörrvakt$|^ordningsvakt$|^badvakt$|^ronderare$|^skyddsvakt$|^kontrollant$|^övervakare$|^kurator$|^coach$|^instruktör$|^läxhjälpare$|^karriärcoach$|^livscoach$|^livskamrat$|^brorson$|^systerson$|^systerdotter$|^brorsdotter$|^svärfar$|^svärmor$|^svärson$|^svärdotter$|^svåger$|^svägerska$|^fosterbarn$|^fosterfar$|^fostermor$|^fosterbror$|^fostersyster$|^närstående$|^bekant$|^medborgare$|^civilsamhällesaktör$|^företagsledare$|^folkvald$|^tjänsteman$|^diplomathustru$|^presidentkandidat$|^parlamentariker$|^hövding$|^stamledare$|^krigare$|^försvarare$|^fredsmäklare$|^förhandlare$|^lobbyist$|^talesman$|^presstalesman$|^PR-expert$|^kommunikatör$|^marknadsförare$|^konsultchef$|^säkerhetsrådgivare$|^brandingenjör$|^riskanalytiker$|^forskningsassistent$|^doktorand$|^magisterstudent$|^stipendiat$|^trainee$|^praktikhandledare$|^karriärvägledare$|^psykologiprofessor$|^sociolog$|^antropolog$|^arkivarie$|^kulturhistoriker$|^museichef$|^konservator$|^teolog$|^religionshistoriker$|^lingvist$|^språkvetare$|^översättare$|^tolk$|^fonetiker$|^logoped$|^retoriker$|^dramaturg$|^manusförfattare$|^regissör$|^filmproducent$|^biografägare$|^ljusdesigner$|^kostymör$|^scenograf$|^teaterkritiker$|^kulturjournalist$|^litteraturkritiker$|^recensent$|^bildredaktör$|^nyhetsankare$|^krönikör$|^spökskrivare$|^ledarskribent$|^debattör$|^talarcoach$|^pr-expert$|^eventkoordinator$|^värdepappersmäklare$|^börsanalytiker$|^försäkringsrådgivare$|^pensionsrådgivare$|^finansiellrådgivare$|^bankdirektör$|^kreditgivare$|^lånehandläggare$|^borgensman$|^panträttsanalytiker$|^fastighetsförvaltare$|^fastighetsmäklare$|^byggprojektledare$|^stadsplanerare$|^inredningsarkitekt$|^landskapsarkitekt$|^trädgårdsarkitekt$|^byggnadsingenjör$|^byggnadskonstruktör$|^fastighetsskötare$|^underhållstekniker$|^driftchef$|^energiexpert$|^hållbarhetsstrateg$|^klimatforskare$|^miljökonsult$|^ekosystemforskare$|^agronom$|^skogsforskare$|^växtbiolog$|^djurvårdare$|^veterinär$|^djursjukvårdare$|^djurskötare$|^djuruppfödare$|^zoopedagog$|^djurtränare$|^jockey$|^ryttare$|^ridlärare$|^hovslagare$|^hästskötare$|^dressyrryttare$|^galopptränare$|^agilitytränare$|^hundtränare$|^kattuppfödare$|^akvarist$|^reptiluppfödare$|^fågeluppfödare$|^fiskodlare$|^biodlare$|^honungsproducent$|^fruktodlare$|^grönsaksodlare$|^spannmålsbonde$|^spannmålsproducent$|^lantbrukare$|^jordbrukare$|^trädgårdsmästare$|^blomsterodlare$|^floristikexpert$|^blomsterhandlare$|^landskapsingenjör$|^parkförvaltare$|^parkvakt$|^trädgårdsarbetare$|^landskapsarkitekt$|^kyrkvaktmästare$|^kyrkogårdsvakt$|^krematoriearbetare$|^gravgrävare$|^ceremonimästare$|^begravningspräst$|^sorgekoordinator$|^bröllopskoordinator$|^vigselförrättare$|^festfixare$|^eventplanerare$|^konferensvärd$|^kongressorganisatör$|^mässarrangör$|^produktionsledare$|^produktchef$|^produktutvecklare$|^kvalitetsansvarig$|^innovationsledare$|^forskningschef$|^forskningsingenjör$|^laboratoriechef$|^laboratorieassistent$|^kemist$|^biokemist$|^molekylärbiolog$|^genetikforskare$|^bioinformatiker$|^biostatistiker$|^kliniskforskare$|^läkemedelsforskare$|^toxikolog$|^farmaceut$|^apotekare$|^receptarie$|^läkemedelskonsulent$|^läkemedelsinformatör$|^klinikchef$|^vårdavdelningschef$|^vårdadministratör$|^sjukhusdirektör$|^sjukvårdsplanerare$|^vårdkoordinator$|^operationssjuksköterska$|^operationsläkare$|^anestesiläkare$|^narkosläkare$|^narkossköterska$|^operationsassistent$|^akutsjuksköterska$|^akutläkare$|^intensivvårdssjuksköterska$|^intensivvårdsläkare$|^vårdpersonal$|^sjukhemschef$|^äldreomsorgschef$|^hemtjänstpersonal$|^personligassistent$|^funktionsstödsassistent$|^stödperson$|^anhörigvårdare$|^omsorgspersonal$|^dagispersonal$|^barnskötare$|^fritidspedagog$|^förskollärare$|^montessoripedagog$|^speciallärare$|^specialpedagog$", re.IGNORECASE)
    for id, sentence in data.items():
      for token in sentence:

        #Alla ord som är ifl oavsett kontext
        if token[1] == "autist" or token[1] == "autists" or token[1] == "autisten" or token[1] == "autister" or token[1] == "autisters" or token[1] == "autisterna" or token[1] == "autisternas" or token[1] == "autisters":
          IFL_sents[id] = sentence
          continue

        #på spektrat är ifl
        if token[1] == "spektrumet" or token[1] == "autismspektrumet" or token[1] == "spektrat" or token[1] == "spektret":
          if sentence[int(token[6])][1].lower() == "på": #Ordets huvud
            IFL_sents[id] = sentence
            continue

        #Om ordet är autistiska/autistisk låt användaren manuellt kolla om det är ifl
        if IFL_regx.match(token[1].lower()) and token[1] != "autistiskt":
          if token != sentence[-1] and Human_regx.match(sentence[int(token[0])][2]):
            IFL_sents[id] = sentence
          else:
            manual_sents[id] = sentence

    for id, sentence in manual_sents.items():
      #print(len(manual_sents))
      test = " ". join(sublist[1] for sublist in sentence)
      print(test)
      question = input("Is this ifl (y/n): ")
      if question.lower() == "y":
        IFL_sents[id] = sentence

    return IFL_sents

####

def main(file, lang):
  lang = lang.lower()
  analyzer = SemanticAnalyzer(file_path= file, lang=lang)
  if lang.lower() == "pfl":
    matches = analyzer.PFL_sentence_matches
  elif lang.lower() == "ifl":
    matches = analyzer.IFL_sentence_matches
  else:
    print("Invalid language choice. Please choose either 'PFL' or 'IFL'.")
    return

  for id, sentence in matches.items():
    print(f"Sentence ID: {int(id) + 1} | {' '.join(token[1] for token in sentence)}") #{' '.join(token[1] for token in sentence)}
    print("")

  print(f"Total {lang} matching sentences: {len(matches)}")
"""
if __name__ == "__main__":
    #file_path = input("Enter the file path: ").lower()
    #if ".conllu" not in file_path:
      #tag_and_convert(file_path)
      #file_path = tag_and_convert(file_path)
    #lang = input("Enter the language (PFL or IFL): ")

    #root_path = r'/content/Project9_data/blogstestfiles'
    #for filename in os.listdir(root_path):
      #filepath = os.path.join(root_path, filename)
      #print(f"Testing {filename}:")
      #main(filepath, "ifl")
      #print("\n")
      #main(filepath, "pfl")
      #print("--------------------")

    filepath = r"/content/Project9_data/autisterna om kvinnor på spektrat.conllu"
    print(f"Testing {filepath}:")
    #main(filepath, "ifl")
    #print("\n")
    main(filepath, "pfl")
    print("--------------------")


SyntaxError: incomplete input (<ipython-input-5-21fb7a43f5c6>, line 99)

In [16]:
def token_counter(path):

  directory = path

  for filename in os.listdir(directory):
    if filename.endswith('.conllu'):
      filepath = os.path.join(directory, filename)
      print("Checking length of: ", filename)
      doc = CoNLL.conll2doc(filepath)
      print(len(doc.sentences))
      #tokens =  len(list(doc.sents))
      #print(tokens)



if __name__ == "__main__":
    filepath = r"/content/Project9_data/CONLLU TEXTS/avhandlingar"
    print(f"Testing {filepath}:")
    token_counter(filepath)

Testing /content/Project9_data/CONLLU TEXTS/avhandlingar:
Checking length of:  well-being in relation to activities in adults with high functioning autism.conllu
1222
Checking length of:  stöd för personer med autismspektrumtillstånd vid hälso- och sjukvårdsbesök.conllu
2505
Checking length of:  autismspecifik kompetens gör skillnad lena holmberg.conllu
791
Checking length of:  tjejer med autism svårigheter och möjligheter i skolan.conllu
614
Checking length of:  effekten av fysisk aktivitet och träning på motoriska och kognitiva förmågor hos barn med autism.conllu
722
Checking length of:  lekens betydelse för den sociala utvecklingen hos barn med autism.conllu
737
Checking length of:  hundens betydelse för klienter med autism.conllu
770
Checking length of:  läsförmåga hos elever i årskurs 3 och 5 samt hos elever i årskurs 5 med autismspektrumtillstånd.conllu
1273
Checking length of:  riskfaktorer för karies hos barn och ungdomar med autismspektrumtillstånd.conllu
668
Checking length o

In [ ]:
"""
import os
from google.colab import files

# Define the directory path
directory = '/content/Project9_data/Blogs'
outdir = '/content/Project9_data/Testfiles'

# Iterate through files in the directory
for folder in os.listdir(directory):
  for filename in os.listdir(os.path.join(directory, folder)):
    folderpath = os.path.join(directory, folder)
    filepath = os.path.join(folderpath, filename)
    print("Converting and downloading: ", filename)
    files.download(os.path.join(outdir, tag_and_convert(filepath)))
"""